# Assignment 2

**Name:** Wendy Stefany Escamilla Valadez

**e-mail:** wendy.escamilla@cucei.udg.mx

______________________________

# MODULES

In [14]:
import math
import numpy as np 
import pandas as pd
import plotly.graph_objects as go
import scipy.stats as stats

from scipy.spatial import distance
from scipy.stats  import wrapcauchy
from scipy.stats import levy_stable


In [15]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

____________________

## Path length - (BM1 vs BM2 vs CRW)

- Write a function that returns a **Brownian Motion (BM)** trajectory in **pandas** df.
- Write a function that returns a **Correlated Random Walk (CRW)** trajectory in **pandas** df.
- Write a function that returns the **path length** for a given trajectory.
- **Compare** at least the **path length** of **three** trajectories as shown in the figure below.
- Display the results using **plotly**.

In [16]:
def BrownianMotionTrayectory(n_steps, s_pos, vel):

    velocity = Vec2d(vel, s_pos[1])

    BM_df = pd.DataFrame(columns=['x', 'y'])
    df_temp = pd.DataFrame([{'x':s_pos[0], 'y':s_pos[1]}]) # DF temp
    BM_df = pd.concat([BM_df,df_temp], ignore_index=True)

    # Generate the trajectory
    for i in range(1,n_steps):
        turn_angle = np.random.choice([0, np.pi/2, np.pi, 3*np.pi/2])

        velocity = velocity.rotated(turn_angle)

        df_temp = pd.DataFrame([{'x':BM_df.x[i-1]+velocity.x, 'y':BM_df.y[i-1]+velocity.y}])
        BM_df = pd.concat([BM_df, df_temp], ignore_index=True)
    
    return BM_df
    

In [17]:
def CorrelatedRandomWalk(n_steps, s_pos, vel, coefficient=0.4):
    velocity = Vec2d(vel, s_pos[1])
    rotations = wrapcauchy.rvs(loc=0, c=coefficient, size=n_steps)

    CRW_df = pd.DataFrame(columns=['x', 'y'])
    # DF temp
    df_temp = pd.DataFrame([{'x':s_pos[0], 'y':s_pos[1]}])
    CRW_df = pd.concat([CRW_df,df_temp], ignore_index=True)

    for i in range(1, n_steps):

        turn_angle = rotations[i]
        # Rotating the velocity vector
        velocity = velocity.rotated(turn_angle)

        df_temp = pd.DataFrame([{'x':CRW_df.x[i-1]+velocity.x, 'y':CRW_df.y[i-1]+velocity.y}])
        CRW_df = pd.concat([CRW_df, df_temp], ignore_index=True)
    
    return CRW_df

In [18]:
def DfTrayectory(trayectory_df, n_steps, s_pos):
     # np.cumsum to accumulate the sum of Euclidean distances along the trajectory.
    trayectory = pd.DataFrame({'y': np.cumsum([s_pos[0]] + 
                    [distance.euclidean(trayectory_df.iloc[i-1], trayectory_df.iloc[i]) for i in range(1, n_steps)])})
    return trayectory
    


## Generate trayectoris and plot

In [19]:
# Initial parameters
s_pos = [0,0]
n_steps = 1000

fig = go.Figure()

# BM 3
BM_df = BrownianMotionTrayectory(n_steps, s_pos, vel=3)
BM_trayectory = DfTrayectory(BM_df,n_steps,s_pos)
fig.add_trace(go.Scatter(
    x=BM_trayectory.index,
    y=BM_trayectory.y,
    mode='lines',
    name="path length BM 3",
    showlegend=True
))

# BM 6
BM_df = BrownianMotionTrayectory(n_steps, s_pos, vel=6)
BM_trayectory = DfTrayectory(BM_df,n_steps,s_pos)
fig.add_trace(go.Scatter(
    x=BM_trayectory.index,
    y=BM_trayectory.y,
    mode='lines',
    name="path length BM 6",
    showlegend=True,
    line=dict(color='red', width=10) 
))


# CRW 6
CRW_df = CorrelatedRandomWalk(n_steps, s_pos, vel=6)
CRW_trayectory = DfTrayectory(CRW_df,n_steps,s_pos)
fig.add_trace(go.Scatter(
    x=CRW_trayectory.index,
    y=CRW_trayectory.y,
    mode='lines',
    name="path length CRW 6",
    showlegend=True
))

fig.show()

__________________________________

## Mean Squared Displacement - (BM vs CRW)
- Write a function that returns the **mean squared displacement** for a given trajectory.
- Compare the **mean squared displacement** curves of at least two trajectories of
different kinds, as shown in the figure below.
- Display the results using **plotly**.

In [20]:
def MeanSquaredDisplacement(trayectory_df, n_steps, x='x', y='y'):
    n = len(trayectory_df)-1
    msd = np.array([np.mean((trayectory_df[x].iloc[i:].values - trayectory_df[x].iloc[:-i].values) ** 2 +
                            (trayectory_df[y].iloc[i:].values - trayectory_df[y].iloc[:-i].values) ** 2
                                       ) 
                                       for i in range(1, n)])
    msd_df = pd.DataFrame(msd, columns=['y'])

    return msd_df



In [21]:
# Initial parameters
s_pos = [0,0]
n_steps = 1000

fig = go.Figure()

# BM 6
BM_df = BrownianMotionTrayectory(n_steps, s_pos, vel=6)
BM_trayectory = MeanSquaredDisplacement(BM_df, n_steps)
BM_trayectory
BM_trayectory
fig.add_trace(go.Scatter(
    x=BM_trayectory.index,
    y=BM_trayectory.y,
    mode='lines',
    name="path length BM 6",
    showlegend=True,
))


# CRW 6
CRW_df = CorrelatedRandomWalk(n_steps, s_pos, vel=6, coefficient=0.9)
CRW_trayectory = MeanSquaredDisplacement(CRW_df, n_steps)
fig.add_trace(go.Scatter(
    x=CRW_trayectory.index,
    y=CRW_trayectory.y,
    mode='lines',
    name="path length CRW 6 C=0.9",
    showlegend=True
))

fig.show()

__________________________________

## Turning-angle Distribution - (source dist. vs observed dist.)
- Consider two **CRW** trajectories with different **Cauchy coefficients**.
- Write a **function** that returns the **turning angles** for a given trajectory.
- Compare the observed distribution (**histogram**) to the source distribution (**curve**) for
both trajectories, as shown in the figure below.
- Display the results using **plotly**.

In [22]:
def turning_angles(trajectory):
    turning_angles = []
    for i in range(1, len(trajectory)-1):

        a = np.array([trajectory.x[i-1], trajectory.y[i-1]])
        b = np.array([trajectory.x[i], trajectory.y[i]])
        c = np.array([trajectory.x[i+1], trajectory.y[i+1]])
        
        ab = b - a 
        bc = c - b  
        
        dot_product = np.dot(ab, bc)
        
        ab_norm = np.linalg.norm(ab)
        bc_norm = np.linalg.norm(bc)
        
        # Calculation of the cosine of the angle
        cos_theta = dot_product / (ab_norm * bc_norm + np.finfo(float).eps)
        
        # cosine is in the range [-1, 1]
        cos_theta = np.clip(cos_theta, -1.0, 1.0)
        
        # Angle between vectors
        angle = np.arccos(cos_theta)
        
        # Angle orientation (determined by cross product)
        cross_p = np.cross(ab, bc)
        orient = np.sign(cross_p)
        if orient == 0:
            orient = 1
        
        # Angle adjusted by orientation
        new_angle = angle * orient
        
        turning_angles.append(new_angle)
    
    return turning_angles

In [23]:
s_pos = [0,0]
n_steps = 1000

crw_trajectory_1 = CorrelatedRandomWalk(n_steps, s_pos, vel=6, coefficient=.6)
crw_trajectory_2 = CorrelatedRandomWalk(n_steps, s_pos, vel=6, coefficient=.9)


turning_angles_1 = turning_angles(crw_trajectory_1)
turning_angles_2 = turning_angles(crw_trajectory_2)

# Plot
fig_hist = go.Figure()
fig_hist.add_trace(go.Histogram(
    x=turning_angles_1,
    name='CRW (speed=6, CRW_coefficient=0.6)',
    marker_color='lightsalmon'
))
fig_hist.add_trace(go.Histogram(
    x=turning_angles_2,
    name='CRW (speed=6, CRW_coefficient=0.9)',
    marker_color='lightsteelblue'
))
fig_hist.update_layout(title='Turning Angles Histogram')



x_values = np.linspace(min(turning_angles_1), max(turning_angles_1), 1000)
y_values = np.array([wrapcauchy.pdf(abs(i), 0.6) for i in x_values])

# scale
scaling_factor = 100
y_values *= scaling_factor

fig_hist.add_trace(go.Scatter(
    x=x_values,
    y=y_values,
    mode='lines',
    name='cauchy 0.6',
    line=dict(color='salmon', width=2)
))

x_values = np.linspace(min(turning_angles_2), max(turning_angles_2), 1000)
y_values = np.array([wrapcauchy.pdf(abs(i), 0.9) for i in x_values])

# scale
scaling_factor = 100
y_values *= scaling_factor

fig_hist.add_trace(go.Scatter(
    x=x_values,
    y=y_values,
    mode='lines',
    name='cauchy 0.3',
    line=dict(color='blue', width=2)
))


fig_hist.show()

________________________________________

## Step-length Distribution - (source dist. vs observed dist.)
- Write a function that returns a **Lévy Walk (LW)** trajectory in **pandas** df.
- Consider two **LW** trajectories with different **alpha coefficients**.
- Write a function that restaurants the **step lengths** for a given trajectory.
- Compare the observed distribution (**histogram**) to the source distribution (**curve**) for
both trajectories, as shown in the figure below.
- Display the results using **plotly**.